### Library imports

In [1]:
#Pandas for creating dataframes
import pandas as pd

#Pyshark to capture packets
import pyshark
#For file operarions
import os
import gc
import sys

### Define required attributed needed to extract from capture.

In [2]:
required_keys = ['ip.dst', 'ip.proto', 'tcp.flags.syn', 'tcp.flags.ack']

### Reading packets from pre-captured file using Pyshark

In [3]:
file_cap = pyshark.FileCapture('captures/botnet-capture-20110810-neris.pcap')

### Extraxt IP packets from the captured file for further processing

In [ ]:
%time
#Write extraction result to
base_directory = 'converted/test3/attack_samples/1/'
#Debugger for pyshark
#file_cap.set_debug()
#Attributes for sample size
startTime = 0.0
endTime = 0.0
i = -1
first = True
#Sample Number
init_sample = 1
dfList = []
while(True):
    i += 1
    #Each packet can have multiple layers (e.g eth, ip, tcp etc.). 
    #Combine them in list for a single packet.
    layerList = []
    #pyshark not able to handle AttributeError, AssertionError and KeyError(index values)
    try:
        #Iterate through layer of packet
        for layer in file_cap[i]:
            # Slice Data according to time
            t = float(file_cap[i].sniff_timestamp)
            #Initial setup
            if startTime == 0.0:
                startTime = t
                endTime = startTime + 300 # 300 sec(5 min) is the sample size
                sample = init_sample
                print(sample, startTime, endTime)
            # Write every sample to csv file
            elif t > endTime:
                if not os.path.exists(base_directory):
                    os.makedirs(base_directory)
                dfList.to_csv(base_directory+str(sample))
                #Clear list after writing this would save memory
                startTime = endTime
                endTime = startTime + 300
                sample += 1
                #Reset after writing to file
                first = True
                print(sample, startTime, endTime)
                gc.collect()
            #We need only ip layer only
            if layer._layer_name == 'ip':
                #Layer values are in the form of dictionary. Filter the attributes
                layer_dict = {key:value for key, value in layer._all_fields.items() 
                              if key in required_keys}
                #Create dataframe from dictionary
                layerList.append(pd.DataFrame(layer_dict, index=[0]))
                #Add timestamp and sample number
                layerList.append(pd.DataFrame({'sniff_timestamp':file_cap[i].sniff_timestamp}, 
                                              index=[0]))
                layerList.append(pd.DataFrame({'sample':sample}, index=[0]))
                #Build packet dataframe from layer frames. Its single row dataframe
                cDf = pd.concat(layerList, axis=1);
                if first:
                    dfList = cDf
                    first = False
                else:
                    dfList = dfList.append(cDf, ignore_index=True)
    except (AttributeError, AssertionError) as e:
        continue  #print('Ipv4 packet does not exist')
    except  KeyError:
        break;

#If sample data is not written to file because for frame size
if(sample == init_sample):
    dfList.to_csv(base_directory+str(init_sample))

Wall time: 0 ns
1 1312966900.475792 1312967200.475792
2 1312967200.475792 1312967500.475792
3 1312967500.475792 1312967800.475792
4 1312967800.475792 1312968100.475792
5 1312968100.475792 1312968400.475792
6 1312968400.475792 1312968700.475792
7 1312968700.475792 1312969000.475792
8 1312969000.475792 1312969300.475792
9 1312969300.475792 1312969600.475792
10 1312969600.475792 1312969900.475792
11 1312969900.475792 1312970200.475792
12 1312970200.475792 1312970500.475792
13 1312970500.475792 1312970800.475792
14 1312970800.475792 1312971100.475792
15 1312971100.475792 1312971400.475792
16 1312971400.475792 1312971700.475792
17 1312971700.475792 1312972000.475792
18 1312972000.475792 1312972300.475792
19 1312972300.475792 1312972600.475792
20 1312972600.475792 1312972900.475792
21 1312972900.475792 1312973200.475792
22 1312973200.475792 1312973500.475792
23 1312973500.475792 1312973800.475792
24 1312973800.475792 1312974100.475792
25 1312974100.475792 1312974400.475792
26 1312974400.4757